In [2]:
# Importing Packages
!pip install pulp
import pulp
from pulp import *
import pandas as pd
import numpy as np
import sklearn
from sklearn import *

     |████████████████████████████████| 14.2 MB 4.3 MB/s 


In [4]:
# Model info and name of constraints
Model = LpProblem("Cost-Problem", LpMinimize)
Wr = LpVariable('Wheat_Regular', cat='Integer', lowBound=0)
Wb = LpVariable('Wheat_Butter', cat='Integer', lowBound=0)
Br = LpVariable('Butter_Regular', cat='Integer', lowBound=0)
Bb = LpVariable('Butter_Butter', cat='Integer', lowBound=0)
BSr = LpVariable('Soda_Regular', cat='Integer', lowBound=0)
BSb = LpVariable('Soda_Butter', cat='Integer', lowBound=0)

In [5]:
# Objective Function
obj_func = 2.46*(Wr + Wb) + 4.32*(Br + Bb) + 1.86*(BSr + BSb)
Model +=  obj_func
Model

Cost-Problem:
MINIMIZE
4.32*Butter_Butter + 4.32*Butter_Regular + 1.86*Soda_Butter + 1.86*Soda_Regular + 2.46*Wheat_Butter + 2.46*Wheat_Regular + 0.0
VARIABLES
0 <= Butter_Butter Integer
0 <= Butter_Regular Integer
0 <= Soda_Butter Integer
0 <= Soda_Regular Integer
0 <= Wheat_Butter Integer
0 <= Wheat_Regular Integer

In [6]:
# 2 Types of croissants
Model += Br >= 0.4*(Wr + Br + BSr)
Model += Bb >= 0.6*(Wb + Bb + BSb)

In [7]:
# The most baking soda we can use in our croissant is 25%
Model += BSr <= 0.25*(Wr + Br + BSr)
Model += BSb <= 0.25*(Wb + Bb + BSb)

In [8]:
# We have already purchased 23 kg butter that must go in our croissants.
Model += (Br + Bb) >= 23

In [9]:
# We have a demand for 350 regular croissants and 500 butter croissants.
Model += (Wr + Br + BSr ) == 350*.05
Model += (Wb + Bb + BSb) == 500*.05

In [10]:
# Total availability
Model += (Wr + Wb) <= 20
Model += (Br + Bb) <= 30 
Model += (BSr + BSb) <=17

In [11]:
# Print Model
Model

Cost-Problem:
MINIMIZE
4.32*Butter_Butter + 4.32*Butter_Regular + 1.86*Soda_Butter + 1.86*Soda_Regular + 2.46*Wheat_Butter + 2.46*Wheat_Regular + 0.0
SUBJECT TO
_C1: 0.6 Butter_Regular - 0.4 Soda_Regular - 0.4 Wheat_Regular >= 0

_C2: 0.4 Butter_Butter - 0.6 Soda_Butter - 0.6 Wheat_Butter >= 0

_C3: - 0.25 Butter_Regular + 0.75 Soda_Regular - 0.25 Wheat_Regular <= 0

_C4: - 0.25 Butter_Butter + 0.75 Soda_Butter - 0.25 Wheat_Butter <= 0

_C5: Butter_Butter + Butter_Regular >= 23

_C6: Butter_Regular + Soda_Regular + Wheat_Regular = 17.5

_C7: Butter_Butter + Soda_Butter + Wheat_Butter = 25

_C8: Wheat_Butter + Wheat_Regular <= 20

_C9: Butter_Butter + Butter_Regular <= 30

_C10: Soda_Butter + Soda_Regular <= 17

VARIABLES
0 <= Butter_Butter Integer
0 <= Butter_Regular Integer
0 <= Soda_Butter Integer
0 <= Soda_Regular Integer
0 <= Wheat_Butter Integer
0 <= Wheat_Regular Integer

In [14]:
Model.solve()
# solvers.PULP_CBC_CMD(fracGap=0)
status =  LpStatus[Model.status]
print(status)

Infeasible


In [15]:
# Total Cost
print("Total Cost:", Model.objective.value())
# Decision Variables
for v in Model.variables():
    try:
        print(v.name,"=", v.value())
    except:
        print("error couldnt find value")

Total Cost: 140.955
Butter_Butter = 15.0
Butter_Regular = 8.0
Soda_Butter = 6.25
Soda_Regular = 4.375
Wheat_Butter = 3.75
Wheat_Regular = 5.125


In [16]:
# Quantity of Butter, Soda and Wheat
print("Butter: ", Bb.value() + Br.value())
print("Soda: ", BSb.value() + BSr.value())
print("Wheat: ", Wb.value() + Wr.value())

Butter:  23.0
Soda:  10.625
Wheat:  8.875
